In [60]:
import os, glob
import numpy as np
from PIL import ImageFile,Image

ImageFile.LOAD_TRUNCATED_IMAGES = True #大きな画像もダウンロード可能にする

classes = ["suzu", "alice"]
num_classes = len(classes)
image_size = 64
num_testdata = 25

X_train = []
X_test  = []
y_train = []
y_test  = []

for index, classlabel in enumerate(classes):
    photos_dir = "./photo/face/" + classlabel
    files = glob.glob(photos_dir + "/*.jpg")
    for i, file in enumerate(files):
        image = Image.open(file)
        image = image.convert("RGB")
        image = image.resize((image_size, image_size))
        data = np.asarray(image)
        if i < num_testdata:
            X_test.append(data)
            y_test.append(index)
        else:

            # angleに代入される値
            # -20
            # -15
            # -10
            #  -5
            # 0
            # 5
            # 10
            # 15
            for angle in range(-20, 20, 5):

                img_r = image.rotate(angle)
                data = np.asarray(img_r)
                X_train.append(data)
                y_train.append(index)
                # FLIP_LEFT_RIGHT　は 左右反転
                img_trains = img_r.transpose(Image.FLIP_LEFT_RIGHT)
                data = np.asarray(img_trains)
                X_train.append(data)
                y_train.append(index)

X_train = np.array(X_train)
X_test  = np.array(X_test)
y_train = np.array(y_train)
y_test  = np.array(y_test)

xy = (X_train, X_test, y_train, y_test)
np.save("./suzu_ali.npy", xy)

In [62]:
import tensorflow as tf
#from keras.models import Sequential
#from keras.layers import Conv2D, MaxPooling2D
#from keras.layers import Activation, Dropout, Flatten, Dense
#from keras.utils import np_utils
#from keras.callbacks import EarlyStopping
#import keras
import numpy as np

classes = ["suzu", "alice"]
num_classes = len(classes)
image_size = 64

"""
データを読み込む関数
"""
def load_data():
    X_train, X_test, y_train, y_test = np.load("./suzu_ali.npy", allow_pickle=True)
    # 入力データの各画素値を0-1の範囲で正規化(学習コストを下げるため)
    X_train = X_train.astype("float") / 255
    X_test  = X_test.astype("float") / 255
    # to_categorical()にてラベルをone hot vector化
    y_train = tf.keras.utils.to_categorical(y_train, num_classes)
    y_test  = tf.keras.utils.to_categorical(y_test, num_classes)

    return X_train, y_train, X_test, y_test

"""
モデルを学習する関数
"""
def train(X, y, X_test, y_test):
    model = tf.keras.models.Sequential()

    model.add(tf.keras.layers.Conv2D(32,(3,3), padding='same',input_shape=X.shape[1:]))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.Conv2D(32,(3,3)))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    model.add(tf.keras.layers.Dropout(0.1))

    model.add(tf.keras.layers.Conv2D(64,(3,3), padding='same'))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.Conv2D(64,(3,3)))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    model.add(tf.keras.layers.Dropout(0.25))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(512))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.Dropout(0.45))
    model.add(tf.keras.layers.Dense(2))
    model.add(tf.keras.layers.Activation('softmax'))

    # https://keras.io/ja/optimizers/
    # 今回は、最適化アルゴリズムにRMSpropを利用
    opt = tf.keras.optimizers.RMSprop(lr=0.00005, decay=1e-6)
    # https://keras.io/ja/models/sequential/
    model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
    
    #過学習を防ぐ
    early_stopping =  tf.keras.callbacks.EarlyStopping(monitor='val_loss',min_delta=0.0,patience=2,)
    
    model.fit(X, y, batch_size=28, epochs=40, validation_data=(X_test,y_test), callbacks=[early_stopping])
    # HDF5ファイルにKerasのモデルを保存
    model.save('./cnn.h5')

    return model

"""
メイン関数
データの読み込みとモデルの学習を行います。
"""

def main():
    # データの読み込み
    X_train, y_train, X_test, y_test = load_data()

    # モデルの学習
    model = train(X_train, y_train, X_test, y_test)

main()

Epoch 1/40
97/97 [==============================] - 20s 208ms/step - loss: 0.6469 - accuracy: 0.6124 - val_loss: 0.7082 - val_accuracy: 0.5000
Epoch 2/40
97/97 [==============================] - 20s 207ms/step - loss: 0.5703 - accuracy: 0.7104 - val_loss: 0.6020 - val_accuracy: 0.7000
Epoch 3/40
97/97 [==============================] - 20s 205ms/step - loss: 0.5197 - accuracy: 0.7592 - val_loss: 0.6058 - val_accuracy: 0.6800
Epoch 4/40
97/97 [==============================] - 20s 208ms/step - loss: 0.4783 - accuracy: 0.7807 - val_loss: 0.5307 - val_accuracy: 0.7600
Epoch 5/40
97/97 [==============================] - 20s 205ms/step - loss: 0.4331 - accuracy: 0.8077 - val_loss: 0.4951 - val_accuracy: 0.8000
Epoch 6/40
97/97 [==============================] - 20s 209ms/step - loss: 0.3878 - accuracy: 0.8439 - val_loss: 0.5571 - val_accuracy: 0.7600
Epoch 7/40
97/97 [==============================] - 20s 209ms/step - loss: 0.3396 - accuracy: 0.8650 - val_loss: 0.4669 - val_accuracy: 0.8200

In [73]:
y_test=[[1,2,3,0],[1,3,2,2]]
y_test  = tf.keras.utils.to_categorical(y_test, 4)
print(y_test)

[[[0. 1. 0. 0.]
  [0. 0. 1. 0.]
  [0. 0. 0. 1.]
  [1. 0. 0. 0.]]

 [[0. 1. 0. 0.]
  [0. 0. 0. 1.]
  [0. 0. 1. 0.]
  [0. 0. 1. 0.]]]
